In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import split
import pandas as pd
import matplotlib.pyplot as plt
import upsetplot
from upsetplot import UpSet
#créer une session dans le master
'''spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-complete") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()'''
spark = SparkSession.builder.master("local").appName("WDC-complete").getOrCreate()

spark.conf.set("spark.worker.cleanup.enabled",True)
spark.conf.set("spark.worker.cleanup.interval",1800)
spark.conf.set("spark.worker.cleanup.appDataTtl",3600)
spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

In [ ]:
# read = spark.read.option("header", True).csv(f"s3a://test-out/{repertory}/hashdom{i}-{parts}")
#read = spark.read.option("header", True).csv(f"s3a://test-out/types/covavevolutionresult")

#Utiliser le read ci-dessous pour afficher aussi les types intangibles
read = spark.read.option("header", True).csv(f"s3a://test-out/types/evolutionPerTypefixed")

read.show(truncate=110)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import col
import json


DF_final = read.withColumn("percentage_count_evolution", col("percentage_count_evolution").cast("double")).fillna(0, subset=["percentage_count_evolution"])

# enlever les types intangibles du dataframe
# Charger le fichier JSON dans une structure de données Python
with open('intangiblesToFilterOut.json', 'r') as json_file:
    values_to_exclude = json.load(json_file)



# Filtrer les lignes en fonction des valeurs présentes dans le fichier JSON
DF_final = DF_final.filter(~col("type").isin(values_to_exclude))

# dans le dataframe des résultats , remplace les valeurs null des colonne "percentage_count_evolution","percentage_average_evolution" et "percentage_coverage_evolution" par 0
DF_final = DF_final.withColumn("percentage_average_evolution", col("percentage_average_evolution").cast("double")).fillna(0, subset=["percentage_average_evolution"])
DF_final = DF_final.withColumn("percentage_coverage_evolution", col("percentage_coverage_evolution").cast("double")).fillna(0, subset=["percentage_coverage_evolution"])
DF_final = DF_final.withColumn("count_after", col("count_after").cast("double")).fillna(0, subset=["count_after"])
DF_final = DF_final.withColumn("count_before", col("count_before").cast("double")).fillna(0, subset=["count_before"])
DF_final = DF_final.withColumn("coverage_after", col("coverage_after").cast("double")).fillna(0, subset=["coverage_after"])
DF_final = DF_final.withColumn("coverage_before", col("coverage_before").cast("double")).fillna(0, subset=["coverage_before"])
DF_final = DF_final.withColumn("average_after", col("average_after").cast("double")).fillna(0, subset=["average_after"])
DF_final = DF_final.withColumn("average_before", col("average_before").cast("double")).fillna(0, subset=["average_before"])
DF_final = DF_final.withColumn("quality_before", col("quality_before").cast("double")).fillna(0, subset=["quality_before"])
DF_final = DF_final.withColumn("quality_after", col("quality_after").cast("double")).fillna(0, subset=["quality_after"])
DF_final = DF_final.withColumn("percentage_quality_evolution", col("percentage_quality_evolution").cast("double")).fillna(0, subset=["percentage_quality_evolution"])

# Filtrer les lignes pour ne garder que les types ayant 10000 occurences au moins en 2022
#DF_final = DF_final.filter(col("count_after") >= 10000.0)
DF_final = DF_final.filter(col("count_before") >= 10000.0)

# on trie les types par nombre d'occurences en 2022
print("top types par nombre d'occurences en 2022")
orderedDF_by_count_2022 = DF_final.orderBy(col('count_before').desc())
#orderedDF_by_count_2022.show()

# on trie les types par nombre d'occurences en 2023
print("top types par nombre d'occurences en 2023")
orderedDF_by_count = DF_final.orderBy(col('count_after').desc())
#orderedDF_by_count.show()


# on trie les types par coverage en 2022
print("top types par coverage en 2022")
orderedDF_by_coverage_2022 = DF_final.orderBy(col('coverage_before').desc())
#orderedDF_by_coverage_2022.show()

# coverage de 1 en 2022
print("coverage de 1 en 2022")
coverage_2022 = orderedDF_by_coverage_2022.filter(orderedDF_by_coverage_2022.coverage_before == 1.0)

# on trie les types par coverage en 2023
print("top types par coverage en 2023")
orderedDF_by_coverage_2023 = DF_final.orderBy(col('coverage_after').desc())

# coverage de 1 en 2023
print("coverage de 1 en 2023")
coverage_2023 = orderedDF_by_coverage_2023.filter(orderedDF_by_coverage_2023.coverage_after == 1.0)


# on trie les types par average en 2022
print("top types par average en 2022")
orderedDF_by_average_2022 = DF_final.orderBy(col('average_before').desc())
#orderedDF_by_average_2022.show(truncate=110)

# on trie les types par average en 2023
print("top types par average en 2023")
orderedDF_by_average_2023 = DF_final.orderBy(col('average_after').desc())
#orderedDF_by_average_2023.show(truncate=110)

# on trie les types par l'évolution de leur count
print("top types par count évolution")
orderedDF_by_percentage_count_evolution = DF_final.orderBy(col('percentage_count_evolution').desc())
#orderedDF_by_percentage_count_evolution.show(truncate=110)

# on trie les types par l'évolution de leur coverage
print("top types par coverage évolution")
orderedDF_by_percentage_coverage_evolution = DF_final.orderBy(col('percentage_coverage_evolution').desc())
#orderedDF_by_percentage_coverage_evolution.show(truncate=110)

# on trie les types par l'évolution de leur average
print("top types par average évolution")
orderedDF_by_percentage_average_evolution = DF_final.orderBy(col('percentage_average_evolution').desc())
#orderedDF_by_percentage_average_evolution.show(truncate=110)

# on trie les types par leur qualité en 2022
print("top types par qualité en 2022")
orderedDF_by_quality_2022 = DF_final.orderBy(col('quality_before').desc())
orderedDF_by_quality_2022.show(truncate=110)

# on trie les types par leur qualité en 2023
print("top types par qualité en 2023")
orderedDF_by_quality_2023 = DF_final.orderBy(col('quality_after').desc())
#orderedDF_by_quality_2023.show(truncate=110)

# on trie les types par l'évolution de leur qualité
print("top types par évolution de leur qualité")
orderedDF_by_quality_evolution = DF_final.orderBy(col('percentage_quality_evolution').desc())
#orderedDF_by_quality_evolution.show(truncate=110)


df_pd_coverage_2022 = orderedDF_by_coverage_2022.limit(10).toPandas()
df_pd_coverage_2022 = df_pd_coverage_2022[::-1]


#Graphique affichant les 10 types de schema.org ayant le meilleur coverage en 2022
ax_coverage=df_pd_coverage_2022.plot(kind='barh' , x='type', y ='coverage_before')
ax_coverage.set_title('Top 10 types with the best coverage in 2022')
ax_coverage.legend_ = None
plt.show()

'''

df_pd_coverage_2023 = orderedDF_by_coverage_2023.limit(10).toPandas()


#Graphique affichant les 10 types de schema.org ayant le meilleur coverage en 2023
ax_coverage=df_pd_coverage_2023.plot(kind='barh' , x='type', y ='coverage_after')
ax_coverage.set_title('Top 10 types with the best coverage in 2023')
ax_coverage.legend_ = None
plt.show()


df_pd_coverage_for_most_used_types_2022 = orderedDF_by_count_2022.limit(10).toPandas()
# on change l'ordre des lignes du dataframe pour afficher correctement les données dans le diagramme
df_pd_coverage_for_most_used_types_2022 = df_pd_coverage_for_most_used_types_2022[::-1]

'''
#Graphique affichant le coverage des 10 types de schema.org les plus utilisés en 2022
ax_coverage=df_pd_coverage_for_most_used_types_2022.plot(kind='barh', x='type', y ='coverage_before')
ax_coverage.set_title('Coverage of the top 10 most used types in 2022')
ax_coverage.legend_ = None
plt.show()

df_pd_coverage_for_most_used_types_2023 = orderedDF_by_count.limit(10).toPandas()
'''

#Graphique affichant le coverage des 10 types de schema.org les plus utilisés en 2023
ax_coverage=df_pd_coverage_for_most_used_types_2023.plot(kind='barh' , x='type', y ='coverage_after')
ax_coverage.set_title('Coverage of the top 10 most used types in 2023')
ax_coverage.legend_ = None
plt.show()
'''


'''
'''
df_pd_count_2023 = orderedDF_by_count.limit(15).toPandas()

print("df_pd_count_2023 ",df_pd_count_2023)
'''

#Graphique affichant l'évolution du coverage des 10 types de schema.org les plus utilisés entre 2022 et 2023
colors = ['green' if (x >= 0) else 'red' for x in df_pd_count_2023['percentage_coverage_evolution']]
ax_coverage=df_pd_count_2023.plot(kind='bar' , x='type', y ='percentage_coverage_evolution', width = 0.5, color=colors)
ax_coverage.set_title('Coverage evolution in percentage of the top 10 most used types in 2022 and 2023')
ax_coverage.legend_ = None
plt.show()


'''
df_pd_average_2022 = orderedDF_by_average_2022.limit(10).toPandas()
#print("df_pd_average_2022 ",df_pd_average_2022)

df_pd_average_2022 = df_pd_average_2022[::-1]

#Graphique affichant les 10 types de schema.org ayant le meilleur average en 2022
ax_average=df_pd_average_2022.plot(kind='barh' , x='type', y ='average_before')
ax_average.set_title('10 highest average properties count')
ax_average.legend_ = None
plt.xlabel('Average properties count')
plt.show()
'''

df_pd_average_2023 = orderedDF_by_average_2023.limit(10).toPandas()
#print("df_pd_average_2023 ",df_pd_average_2023)


#Graphique affichant les 10 types de schema.org ayant le meilleur average en 2023
ax_average=df_pd_average_2023.plot(kind='barh' , x='type', y ='average_after')
ax_average.set_title('Top 10 most structured types with the best average in 2023')
ax_average.legend_ = None
plt.show()


df_pd_count_2022 = orderedDF_by_count_2022.limit(10).toPandas()
# on change l'ordre des lignes du dataframe pour afficher correctement les données dans le diagramme
df_pd_count_2022 = df_pd_count_2022[::-1]
#print("df_pd_count_2022 ",df_pd_count_2022)

#Graphique affichant les 10 types de schema.org les plus utilisés en 2022 avec leur average
ax_average=df_pd_count_2022.plot(kind='barh', x='type', y ='average_before')
ax_average.set_title('Average of the top 10 most used types')
#ax_count.legend('coverage')
ax_average.legend_ = None
plt.xlabel('Average properties count')
plt.show()


'''
df_pd_count = orderedDF_by_count_2022.limit(15).toPandas()
print("df_pd_count ",df_pd_count)
'''

#Graphique affichant les 10 types de schema.org les plus utilisés en 2023 avec leur average
ax_average=df_pd_count.plot(kind='barh', x='type', y ='average_after')
ax_average.set_title('Top 10 most structured types with it average in 2023')
#ax_count.legend('coverage')
ax_average.legend_ = None
plt.show()

df_pd_count = orderedDF_by_count.limit(10).toPandas()
print("df_pd_count ",df_pd_count)

#Graphique affichant les 10 types de schema.org les plus utilisés en 2023 avec le pourcentage d'évolution de leur average entre 2022 et 2023
colors = ['green' if (x >= 0) else 'red' for x in df_pd_count_2023['percentage_average_evolution']]
ax_average=df_pd_count.plot(kind='bar', x='type', y ='percentage_average_evolution', width = 0.5, color=colors)
ax_average.set_title('Average evolution (in percentage) of top 10 most structured types with it average in 2023')
#ax_count.legend('coverage')
ax_average.legend_ = None
plt.show()

'''

#Graphique affichant les 15 types de schema.org les plus utilisés en 2022 avec leur nombre d'occurences en 2022 et en 2023
ax_count=df_pd_count.plot.bar(x='type', y =['count_before','count_after'],width=0.9)
#ax_count=df_pd_count.plot.bar(x='type', y =['count_after'],width=0.1)

ax_count.set_title('Top 15 most used types in 2022 with the count of 2022 and 2023')
ax_count.legend(['count 2022','count 2023'])
plt.show()

#Graphique affichant les 15 types de schema.org les plus utilisés en 2022 avec le pourcentage d'évolution de leur nombre d'occurences entre 2022 et 2023
#ax_count=df_pd_count.plot.bar(x='type', y =['count_after'],width=0.1)
colors = ['green' if (x >= 0) else 'red' for x in df_pd_count['percentage_count_evolution']]
ax_count=df_pd_count.plot(kind='bar' , x='type', y ='percentage_count_evolution', width = 0.5, color=colors)
ax_count.set_title('Count evolution between 2022 and 2023 for Top 15 most used types')
ax_count.legend(['count evolution in percentage'])
plt.show()


#Graphique affichant les 15 types de schema.org les plus utilisés en 2022 avec leur average en 2022 et en 2023
ax_count=df_pd_count.plot.bar(x='type', y =['average_before','average_after'],width=0.9)

ax_count.set_title('Top 15 most used types in 2022 with the average of 2022 and 2023')
ax_count.legend(['average 2022','average 2023'])
plt.show()

#Graphique affichant les 15 types de schema.org les plus utilisés en 2022 avec le pourcentage d'évolution de leur average entre 2022 et 2023
colors = ['green' if (x >= 0) else 'red' for x in df_pd_count['percentage_average_evolution']]
ax_count=df_pd_count.plot(kind='bar' , x='type', y ='percentage_average_evolution', width = 0.5, color=colors)
ax_count.set_title('Average evolution between 2022 and 2023 for Top 15 most used types')
ax_count.legend(['average evolution in percentage'])
plt.show()


#Graphique affichant les 15 types de schema.org les plus utilisés en 2022 avec leur coverage en 2022 et en 2023
ax_count=df_pd_count.plot.bar(x='type', y =['coverage_before','coverage_after'],width=0.9)

ax_count.set_title('Top 15 most used types in 2022 with the coverage of 2022 and 2023')
ax_count.legend(['coverage 2022','coverage 2023'])
plt.show()

#Graphique affichant les 15 types de schema.org les plus utilisés en 2022 avec le pourcentage d'évolution de leur coverage entre 2022 et 2023
colors = ['green' if (x >= 0) else 'red' for x in df_pd_count['percentage_coverage_evolution']]
ax_count=df_pd_count.plot(kind='bar' , x='type', y ='percentage_coverage_evolution', width = 0.5, color=colors)
ax_count.set_title('Coverage evolution between 2022 and 2023 for Top 15 most used types')
ax_count.legend(['coverage evolution in percentage'])
plt.show()
'''
'''
df_pd_count_evolution = orderedDF_by_percentage_count_evolution.limit(100).toPandas()
df_pd_count_evolution = df_pd_count_evolution[::-1]
print(df_pd_count_evolution)


#Graphique affichant les 15 types de schema.org ayant le meilleur pourcentage d'évolution de leur nombre d'occurence entre 2022 et 2023
ax_average=df_pd_count_evolution.plot(kind='barh', x='type', y ='percentage_count_evolution')
ax_average.set_title('Top 15 types with best count evolution')
ax_average.legend(['count evolution in percentage'])
plt.show()

df_pd_coverage_evolution = orderedDF_by_percentage_coverage_evolution.limit(15).toPandas()
df_pd_coverage_evolution = df_pd_coverage_evolution[::-1]


#Graphique affichant les 15 types de schema.org ayant le meilleur pourcentage d'évolution de leur coverage entre 2022 et 2023
ax_average=df_pd_coverage_evolution.plot(kind='barh', x='type', y ='percentage_coverage_evolution')
ax_average.set_title('Top 15 types with best coverage evolution')
ax_average.legend(['coverage evolution in percentage'])
plt.show()


df_pd_average_evolution = orderedDF_by_percentage_average_evolution.limit(15).toPandas()
df_pd_average_evolution = df_pd_average_evolution[::-1]

#Graphique affichant les 15 types de schema.org ayant le meilleur pourcentage d'évolution de leur average entre 2022 et 2023
ax_average=df_pd_average_evolution.plot(kind='barh', x='type', y ='percentage_average_evolution')
ax_average.set_title('Top 15 types with best average evolution')
ax_average.legend(['average evolution in percentage'])
plt.show()

df_pd_quality_evolution = orderedDF_by_quality_evolution.limit(15).toPandas()
df_pd_quality_evolution = df_pd_quality_evolution[::-1]

#Graphique affichant les 15 types de schema.org ayant le meilleur pourcentage d'évolution de leur qualité entre 2022 et 2023
ax_average=df_pd_quality_evolution.plot(kind='barh', x='type', y ='percentage_quality_evolution')
ax_average.set_title('Top 15 types with best quality evolution')
ax_average.legend(['quality evolution in percentage'])
plt.show()


df_pd_quality_2022 = orderedDF_by_quality_2022.limit(15).toPandas()
df_pd_quality_2022 = df_pd_quality_2022[::-1]


#Graphique affichant les 15 types de schema.org ayant la meilleure qualité en 2022
ax_average=df_pd_quality_2022.plot(kind='barh', x='type', y ='quality_before')
ax_average.set_title('Top 15 types with best quality in 2022')
#ax_average.legend(['average evolution in percentage'])
plt.show()


df_pd_quality_2023 = orderedDF_by_quality_2023.limit(15).toPandas()
df_pd_quality_2023 = df_pd_quality_2023[::-1]

#Graphique affichant les 15 types de schema.org ayant la meilleure qualité en 2023
ax_average=df_pd_quality_2023.plot(kind='barh', x='type', y ='quality_after')
ax_average.set_title('Top 15 types with best quality in 2023')
#ax_average.legend(['average evolution in percentage'])
plt.show()


#Graphique affichant les 15 types de schema.org les plus utilisés en 2022 avec leur qualité en 2022 et en 2023
ax_count=df_pd_count.plot.bar(x='type', y =['quality_before','quality_after'],width=0.9)

ax_count.set_title('Top 15 most used types in 2022 with quality of 2022 and 2023')
ax_count.legend(['quality 2022','quality 2023'])
plt.show()


#Graphique affichant les 15 types de schema.org les plus utilisés en 2022 avec le pourcentage d'évolution de leur qualité entre 2022 et 2023
colors = ['green' if (x >= 0) else 'red' for x in df_pd_count['percentage_quality_evolution']]
ax_count=df_pd_count.plot(kind='bar' , x='type', y ='percentage_quality_evolution', width = 0.5, color=colors)
ax_count.set_title('Quality evolution between 2022 and 2023 for Top 15 most used types')
ax_count.legend(['quality evolution in percentage'])
plt.show()
